In [1]:
polling = read.csv("./data/PollingData.csv")
str(polling)

'data.frame':	145 obs. of  7 variables:
 $ State     : Factor w/ 50 levels "Alabama","Alaska",..: 1 1 2 2 3 3 3 4 4 4 ...
 $ Year      : int  2004 2008 2004 2008 2004 2008 2012 2004 2008 2012 ...
 $ Rasmussen : int  11 21 NA 16 5 5 8 7 10 NA ...
 $ SurveyUSA : int  18 25 NA NA 15 NA NA 5 NA NA ...
 $ DiffCount : int  5 5 1 6 8 9 4 8 5 2 ...
 $ PropR     : num  1 1 1 1 1 ...
 $ Republican: int  1 1 1 1 1 1 1 1 1 1 ...


三个大选年，50个州，因此应该有150个观测值。但只有145个。

In [2]:
table(polling$Year)


2004 2008 2012 
  50   50   45 

发现2012年只有45个观测值。但这问题不大。第二个问题是很多NA值。

In [3]:
summary(polling)

         State          Year        Rasmussen          SurveyUSA       
 Arizona    :  3   Min.   :2004   Min.   :-41.0000   Min.   :-33.0000  
 Arkansas   :  3   1st Qu.:2004   1st Qu.: -8.0000   1st Qu.:-11.7500  
 California :  3   Median :2008   Median :  1.0000   Median : -2.0000  
 Colorado   :  3   Mean   :2008   Mean   :  0.0404   Mean   : -0.8243  
 Connecticut:  3   3rd Qu.:2012   3rd Qu.:  8.5000   3rd Qu.:  8.0000  
 Florida    :  3   Max.   :2012   Max.   : 39.0000   Max.   : 30.0000  
 (Other)    :127                  NA's   :46         NA's   :71        
   DiffCount           PropR          Republican    
 Min.   :-19.000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.: -6.000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :  1.000   Median :0.6250   Median :1.0000  
 Mean   : -1.269   Mean   :0.5259   Mean   :0.5103  
 3rd Qu.:  4.000   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   : 11.000   Max.   :1.0000   Max.   :1.0000  
                                                    

### 缺失值

- 把带缺失值的样本抛弃？那样会抛弃一半的样本。
- 抛弃带缺失值的变量？但希望保留`Rasmussen`和`SurveyUSA`。
- 用均值填充缺失值？
    - 民主党和共和党在总票数上接近，所以`DiffCount`很可能是0。
    - 如果别的民意调查偏向其中一个党，则填充会让这种偏向扩散。
- 用非缺失值填充？
    - `SurveyUSA`的值根据`Rasmussen`的值来填。

用R的插值包实现multiple imputation。

In [6]:
#install.packages("mice", repos = "http://cran.us.r-project.org")

package 'mice' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\CYS\AppData\Local\Temp\RtmpMZpJ8V\downloaded_packages


In [2]:
library("mice")

Loading required package: Rcpp
mice 2.25 2015-11-09


In [3]:
simple = polling[c("Rasmussen", "SurveyUSA", "PropR", "DiffCount")]
summary(simple)

   Rasmussen          SurveyUSA            PropR          DiffCount      
 Min.   :-41.0000   Min.   :-33.0000   Min.   :0.0000   Min.   :-19.000  
 1st Qu.: -8.0000   1st Qu.:-11.7500   1st Qu.:0.0000   1st Qu.: -6.000  
 Median :  1.0000   Median : -2.0000   Median :0.6250   Median :  1.000  
 Mean   :  0.0404   Mean   : -0.8243   Mean   :0.5259   Mean   : -1.269  
 3rd Qu.:  8.5000   3rd Qu.:  8.0000   3rd Qu.:1.0000   3rd Qu.:  4.000  
 Max.   : 39.0000   Max.   : 30.0000   Max.   :1.0000   Max.   : 11.000  
 NA's   :46         NA's   :71                                           

多重插值与split类似，每次的结果都不一样。设定seed，确保结果一致。但一旦执行，kernel就die了。

In [4]:
#set.seed(144)

In [4]:
imputed = complete(mice(simple))
str(imputed)


 iter imp variable
  1   1  Rasmussen  SurveyUSA
  1   2  Rasmussen  SurveyUSA
  1   3  Rasmussen  SurveyUSA
  1   4  Rasmussen  SurveyUSA
  1   5  Rasmussen  SurveyUSA
  2   1  Rasmussen  SurveyUSA
  2   2  Rasmussen  SurveyUSA
  2   3  Rasmussen  SurveyUSA
  2   4  Rasmussen  SurveyUSA
  2   5  Rasmussen  SurveyUSA
  3   1  Rasmussen  SurveyUSA
  3   2  Rasmussen  SurveyUSA
  3   3  Rasmussen  SurveyUSA
  3   4  Rasmussen  SurveyUSA
  3   5  Rasmussen  SurveyUSA
  4   1  Rasmussen  SurveyUSA
  4   2  Rasmussen  SurveyUSA
  4   3  Rasmussen  SurveyUSA
  4   4  Rasmussen  SurveyUSA
  4   5  Rasmussen  SurveyUSA
  5   1  Rasmussen  SurveyUSA
  5   2  Rasmussen  SurveyUSA
  5   3  Rasmussen  SurveyUSA
  5   4  Rasmussen  SurveyUSA
  5   5  Rasmussen  SurveyUSA
'data.frame':	145 obs. of  4 variables:
 $ Rasmussen: int  11 21 19 16 5 5 8 7 10 13 ...
 $ SurveyUSA: int  18 25 22 9 15 3 15 5 8 21 ...
 $ PropR    : num  1 1 1 1 1 ...
 $ DiffCount: int  5 5 1 6 8 9 4 8 5 2 ...


In [5]:
summary(imputed)

   Rasmussen         SurveyUSA           PropR          DiffCount      
 Min.   :-41.000   Min.   :-33.000   Min.   :0.0000   Min.   :-19.000  
 1st Qu.:-10.000   1st Qu.:-11.000   1st Qu.:0.0000   1st Qu.: -6.000  
 Median :  3.000   Median :  1.000   Median :0.6250   Median :  1.000  
 Mean   :  2.228   Mean   :  1.531   Mean   :0.5259   Mean   : -1.269  
 3rd Qu.: 12.000   3rd Qu.: 18.000   3rd Qu.:1.0000   3rd Qu.:  4.000  
 Max.   : 39.000   Max.   : 30.000   Max.   :1.0000   Max.   : 11.000  

此时不再有NA。放回polling中。

In [6]:
polling$Rasmussen = imputed$Rasmussen
polling$SurveyUSA = imputed$SurveyUSA
summary(polling)

         State          Year        Rasmussen         SurveyUSA      
 Arizona    :  3   Min.   :2004   Min.   :-41.000   Min.   :-33.000  
 Arkansas   :  3   1st Qu.:2004   1st Qu.:-10.000   1st Qu.:-11.000  
 California :  3   Median :2008   Median :  3.000   Median :  1.000  
 Colorado   :  3   Mean   :2008   Mean   :  2.228   Mean   :  1.531  
 Connecticut:  3   3rd Qu.:2012   3rd Qu.: 12.000   3rd Qu.: 18.000  
 Florida    :  3   Max.   :2012   Max.   : 39.000   Max.   : 30.000  
 (Other)    :127                                                     
   DiffCount           PropR          Republican    
 Min.   :-19.000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.: -6.000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :  1.000   Median :0.6250   Median :1.0000  
 Mean   : -1.269   Mean   :0.5259   Mean   :0.5103  
 3rd Qu.:  4.000   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   : 11.000   Max.   :1.0000   Max.   :1.0000  
                                                    

### 建模

用2004和2008年的数据训练，用2012年的数据测试。

In [7]:
Train = subset(polling, Year == 2004 | Year == 2008)
Test = subset(polling, Year == 2012)

### 基准模型

最笨的基准模型：无条件地预测共和党会赢。这个基准模型在训练集中的准确率为53%。

In [8]:
table(Train$Republican)


 0  1 
47 53 

稍微聪明的基准模型：`Rasmussen`说谁赢，就预测谁赢。这个模型还是很靠谱的。

In [9]:
table(sign(Train$Rasmussen))


-1  0  1 
42  2 56 

In [10]:
table(Train$Republican, sign(Train$Rasmussen))

   
    -1  0  1
  0 42  1  4
  1  0  1 52

### Logistic模型

需要考虑多重共线性的问题。因为这是很可能存在的。

In [11]:
cor(Train)

ERROR: Error in cor(Train): 'x' must be numeric


In [12]:
str(Train)

'data.frame':	100 obs. of  7 variables:
 $ State     : Factor w/ 50 levels "Alabama","Alaska",..: 1 1 2 2 3 3 4 4 5 5 ...
 $ Year      : int  2004 2008 2004 2008 2004 2008 2004 2008 2004 2008 ...
 $ Rasmussen : int  11 21 19 16 5 5 7 10 -11 -27 ...
 $ SurveyUSA : int  18 25 22 9 15 3 5 8 -11 -24 ...
 $ DiffCount : int  5 5 1 6 8 9 8 5 -8 -5 ...
 $ PropR     : num  1 1 1 1 1 1 1 1 0 0 ...
 $ Republican: int  1 1 1 1 1 1 1 1 0 0 ...


In [13]:
cor(Train[c("Rasmussen", "SurveyUSA", "PropR", "DiffCount", "Republican")])

,Rasmussen,SurveyUSA,PropR,DiffCount,Republican
Rasmussen,1.0000000,0.8952416,0.8352601,0.4984191,0.7877702
SurveyUSA,0.8952416,1.0000000,0.8817534,0.5766725,0.8346488
PropR,0.8352601,0.8817534,1.0000000,0.8273785,0.9484204
DiffCount,0.4984191,0.5766725,0.8273785,1.0000000,0.8092777
Republican,0.7877702,0.8346488,0.9484204,0.8092777,1.0000000


考虑建立只有一个自变量的回归模型。找一个与因变量相关系数最高的变量作为自变量，`PropR`。

In [14]:
mod1 = glm(Republican~PropR, data=Train, family=binomial)
summary(mod1)


Call:
glm(formula = Republican ~ PropR, family = binomial, data = Train)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.22880  -0.06541   0.10260   0.10260   1.37392  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -6.146      1.977  -3.108 0.001882 ** 
PropR         11.390      3.153   3.613 0.000303 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 138.269  on 99  degrees of freedom
Residual deviance:  15.772  on 98  degrees of freedom
AIC: 19.772

Number of Fisher Scoring iterations: 8


看模型在训练集上表现如何。

In [15]:
pred1 = predict(mod1, type="response")
table(Train$Republican, pred1 >= 0.5)

   
    FALSE TRUE
  0    45    2
  1     2   51

考虑有两个自变量的回归模型。要找两个相关系数低的作为自变量，避免多重共线性问题。发现`SurveyUSA`和`DiffCount`的相关性低。

In [21]:
mod2 = glm(Republican~SurveyUSA+DiffCount, data=Train, family=binomial)
pred2 = predict(mod2, type="response")
table(Train$Republican, pred2>=0.5)

   
    FALSE TRUE
  0    45    2
  1     1   52

In [22]:
summary(mod2)


Call:
glm(formula = Republican ~ SurveyUSA + DiffCount, family = binomial, 
    data = Train)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.10984  -0.00717   0.00750   0.04550   1.46490  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)  
(Intercept)  -0.9949     1.2099  -0.822   0.4109  
SurveyUSA     0.3405     0.2049   1.661   0.0966 .
DiffCount     0.7020     0.4201   1.671   0.0947 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 138.27  on 99  degrees of freedom
Residual deviance:  11.61  on 97  degrees of freedom
AIC: 17.61

Number of Fisher Scoring iterations: 9


发现AIC变小了，好像更好了。但显著程度不好。准备用第二个模型进行测试集上的预测。先看基准模型在测试集上的表现。

In [23]:
table(Test$Republican, sign(Test$Rasmussen))

   
    -1  0  1
  0 18  2  4
  1  0  0 21

In [24]:
TestPrediction = predict(mod2, newdata=Test, type="response")
table(Test$Republican, TestPrediction>=0.5)

   
    FALSE TRUE
  0    23    1
  1     0   21

我们犯错在什么地方？

In [25]:
subset(Test, TestPrediction >= 0.5 & Republican == 0)

,State,Year,Rasmussen,SurveyUSA,DiffCount,PropR,Republican
24,Florida,2012,2,0,6,0.6666667,0
